# Satellite Propagation and Measurement Modeling

## State Transition Model

In this section, 1e will be using a two-body Keplerian model to propagate the satellite.
The two-body Keplerian model is based on the two-body model that assumes 
a spherical gravity field for the Earth and neglects third body effects 
and other environmental perturbations.

The two-body Keplerian model is constructing using the poliastro toolbox,
with an example application shown below.

In [1]:
from astropy.time import Time
from poliastro.twobody import Orbit
from poliastro.bodies import Earth
from astropy import units as u
import numpy as np

In [2]:
# Define initial position and velocity vectors
position = [3235.64171524,  2693.72565982, -5335.42793567]
velocity = [-4.87430005,  5.89879341,  0.01977648]

# Create a time object that corresponds to the initial state
t_astropy = Time('2024-05-13T00:00:00', format='isot', scale='utc')
print(t_astropy)

# Create a poliastro Orbit object from position and velocity vectors
orbit = Orbit.from_vectors(Earth, position<< u.km, velocity<< u.km / u.s, epoch=t_astropy)

# Obtain the position and velocity values
print('The position of the satellite is', orbit.r)
print('The velocity of the satellite is', orbit.v)

2024-05-13T00:00:00.000
The position of the satellite is [ 3235.64171524  2693.72565982 -5335.42793567] km
The velocity of the satellite is [-4.87430005  5.89879341  0.01977648] km / s


In [3]:
# Propagate the satellite for 30 minutes by calling the propagate function
orbit_30m = orbit.propagate(30 << u.min)

# Obtain the new position and velocity
print('The new position of the satellite is', orbit_30m.r)
print('The new velocity of the satellite is', orbit_30m.v)

The new position of the satellite is [-5304.15733845  3497.07469551  2374.42056251] km
The new velocity of the satellite is [-1.10089819 -5.34797219  5.38379247] km / s


## Measurement Model

In the section, we will define the measurement model for GPS measurements. We will be directly measuring the the position and velocity of the satellite, i.e, 
$$y = h(x) = [x,y,z,Vx,Vy,Vz]$$

In [4]:
# The ideal GPS measurement is then:
yt0 = np.hstack((orbit.r.value, orbit.v.value))
y30min = np.hstack((orbit_30m.r.value, orbit_30m.v.value))
print("y_{t0} =", yt0)
print("y_{30min} =", y30min)

y_{t0} = [ 3.23564172e+03  2.69372566e+03 -5.33542794e+03 -4.87430005e+00
  5.89879341e+00  1.97764800e-02]
y_{30min} = [-5.30415734e+03  3.49707470e+03  2.37442056e+03 -1.10089819e+00
 -5.34797219e+00  5.38379247e+00]


## Loading the measurement data

In the section, we will laod the measurement data from a file.

In [5]:
# Load measurement data from file
npzfile = np.load('GPS_meas.npz',allow_pickle=True)
print(npzfile.files)

['t_measurements', 'measurements']


In [6]:
# Extract noisy measurements
noisy_measurement = npzfile['measurements']
print(noisy_measurement.shape)

(721, 6)


In [7]:
# Extract measurement timestamps
t_measurement = npzfile['t_measurements']
print(t_measurement.shape)

(721,)


In [8]:
print(noisy_measurement)
print(t_measurement)

[[-1.93725325e+02  5.38460985e+03 -4.14317926e+03 -6.07961179e+00
   2.69462526e+00  3.78864317e+00]
 [-3.34443886e+03  5.75797263e+03 -1.35422632e+03 -4.77433757e+00
  -1.41161891e+00  5.82006204e+00]
 [-5.30833235e+03  3.48893022e+03  2.39893066e+03 -1.08152050e+00
  -5.35787671e+00  5.37338356e+00]
 ...
 [-4.36713006e+03  2.53811710e+03  4.53405086e+03 -7.46081633e-01
  -6.94374665e+00  3.15519570e+00]
 [-3.74356901e+03 -2.08478241e+03  5.25934243e+03  2.64478410e+00
  -7.13246324e+00 -9.48712696e-01]
 [-1.55261882e+03 -5.49331478e+03  3.66634194e+03  4.66664233e+00
  -4.23829542e+00 -4.36753030e+00]]


AttributeError: 'TimeISOT' object has no attribute 'precision'